In [ ]:
import os
from openai import AzureOpenAI
import PyPDF2

In [2]:
# creating a pdf reader object
reader = PyPDF2.PdfReader('crop.pdf')
user_prompt=''
for i in range(len(reader.pages)):
    user_prompt += reader.pages[i].extract_text()

NameError: name 'PyPDF2' is not defined

In [4]:
predicted_crop= "maize"

In [5]:
endpoint = os.getenv("ENDPOINT_URL", "https://sparcopenai.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "spmod1")
OPENAI_API_KEY = "7e4eacc1a88f48c481c1dd2727d3979e"

In [6]:
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_version="2024-05-01-preview",
    api_key=OPENAI_API_KEY
)

In [7]:
# System message
system_message = """
[INST]<<SYS>> You are a data scientist and your job is to summarize ML model output explanation.
The output should strictly be in following sections: 1) "Important Drivers" 2) "Prediction Methodology" 
3) Explain, point wise, why is the {0} as a crop recommended. 
Make sure you replace any logical symbol with plain english words.
<</SYS>>[/INST]
""".format(predicted_crop)

In [8]:
prompt = f"{user_prompt}\n{system_message}"

In [9]:
completion = client.chat.completions.create(
    model=deployment,
    messages= [
    {
      "role": "user",
      "content": prompt
    }],
    max_tokens=3500,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False
)
# print(completion.to_json())

In [10]:
import json

In [11]:
data= completion.to_json()

In [138]:
# print(completion.to_json()['choices'][0]['message']['content'])

In [12]:
data_str = json.loads(data)

# Extracting the "message" content
data_str['choices'][0]['message']['content']

'"Important Drivers"\n\nThe significant drivers in the dataset are humidity, rainfall, K, P, and N. The importance of these features in descending order is as follows: \n\n1. Humidity: 26%\n2. Rainfall: 24%\n3. K: 22%\n4. P: 15%\n5. N: 14%\n\n"Prediction Methodology"\n\nThe prediction is made using decision trees that take into account the features of the data such as humidity, rainfall, K, P, and N. The decision tree divides the data into smaller subsets while at the same time an associated decision tree is incrementally developed. The final result is a tree with decision nodes and leaf nodes. \n\nFor instance, rice is predicted as a crop when the conditions are as follows: \n- Rainfall is greater than 123.59 \n- Humidity is between 80.54 and 89.91 \n- P is between 28.00 and 51.00\n- N is less than or equal to 85.00 \n- K is less than or equal to 49.00 \n\nWhy is rice as a crop recommended?\n\n1. High Rainfall: Rice as a crop requires substantial rainfall. In this case, the rainfall i